In [1]:
from torchvision import models
import torch.nn as nn

def pretrain_model(freeze_percent=0.0, freeze_all_except_last_layer=False, num_classes=10):
    """
    Loads pretrained ResNet50, applies fine-tuning strategy, and replaces the final layer.

    Parameters:
    - freeze_percent (float): Fraction of layers to freeze (0.0 to 1.0).
    - freeze_all_except_last_layer (bool): If True, freezes all layers except the last fully connected layer.
    - num_classes (int): Number of output classes for classification.

    Returns:
    - model (nn.Module): Modified ResNet50 model.
    """
    model = models.resnet50(pretrained=True)

    if freeze_all_except_last_layer:
        for param in model.parameters():
            param.requires_grad = False
        for param in model.fc.parameters():
            param.requires_grad = True
    elif freeze_percent > 0.0:
        all_params = list(model.parameters())
        freeze_until = int(len(all_params) * freeze_percent)
        for i, param in enumerate(all_params):
            param.requires_grad = i >= freeze_until

    # Replace final classification layer
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)

    return model


In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
def data_load(data_dir, batch_size=64, val_split=0.2, data_augmentation=True):
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ]) if data_augmentation else transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])

    val_transforms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])

    dataset = datasets.ImageFolder(root=data_dir, transform=train_transforms)
    val_size = int(val_split * len(dataset))
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    val_dataset.dataset.transform = val_transforms

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader


In [3]:
def test_data_load(test_data_dir, data_augmentation='No'):
    # Transforms for resizing, normalization, etc.
    resize = transforms.Resize((256, 256))
    convert_to_tensor = transforms.ToTensor()
    normalize = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    resize_crop = transforms.RandomResizedCrop(256)
    h_flip = transforms.RandomHorizontalFlip()
    color_jitter = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
    rand_rotation = transforms.RandomRotation(20)

    if data_augmentation == 'Yes':
        transform_img = transforms.Compose([
            resize_crop,
            h_flip,
            color_jitter,
            rand_rotation,
            convert_to_tensor,
            normalize
        ])
    else:
        transform_img = transforms.Compose([
            resize,
            convert_to_tensor,
            normalize
        ])

    # Load dataset
    test_data = ImageFolder(root=test_data_dir, transform=transform_img)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

    return test_loader


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define the device globally
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def model_train_val(model, train_loader, val_loader, epochs=5, learning_rate=1e-3):
    """
    Trains and validates the CNN model.
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for inputs, labels in loop:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            loop.set_postfix(loss=loss.item(), acc=100. * correct / total)

        train_loss /= total
        train_acc = 100. * correct / total

        # Validation Phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]"):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= total
        val_acc = 100. * correct / total

        print(f"\nEpoch {epochs}/{epochs} => "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        
        wandb.log({
            'epoch+1': epochs + 1,
            'Train Loss': train_loss,
            'Train Accuracy': train_acc,
            'Validation Loss': val_loss,
            'Validation Accuracy': val_acc
        })

    print("Training complete!")
    return model  

   

In [5]:
import wandb
wandb.login(key='594642013968a68e466138e783dcece6765c43b9')
# Initialize your W&B run
wandb.init(
    project="DL_Assignment_2",         # Change this to your actual project name
    name="pre_trainedmodel_rs_net",  # Optional: name of this run
    
)
model=pretrain_model(freeze_percent=0.8, freeze_all_except_last_layer=False, num_classes=10)
model=model.to(device)
train_dir = '/kaggle/input/inaturalist/inaturalist_12K/train'
train_loader,val_loader = data_load(train_dir, data_augmentation=True)
trained_model =  model_train_val(model, train_loader, val_loader, epochs=5)
wandb.finish()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bgorai005 (bgorai005-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 165MB/s] 
Epoch 1/5 [Val]: 100%|██████████| 32/32 [00:19<00:00,  1.62it/s]



Epoch 5/5 => Train Loss: 1.0353, Train Acc: 65.40%, Val Loss: 0.9014, Val Acc: 71.59%


Epoch 2/5 [Val]: 100%|██████████| 32/32 [00:12<00:00,  2.54it/s]



Epoch 5/5 => Train Loss: 0.6168, Train Acc: 79.71%, Val Loss: 0.9647, Val Acc: 70.09%


...

Epoch 4/5 [Val]: 100%|██████████| 32/32 [00:12<00:00,  2.53it/s]



Epoch 5/5 => Train Loss: 0.2557, Train Acc: 91.41%, Val Loss: 1.1880, Val Acc: 70.84%


Epoch 5/5 [Val]: 100%|██████████| 32/32 [00:13<00:00,  2.45it/s]


Epoch 5/5 => Train Loss: 0.1791, Train Acc: 94.14%, Val Loss: 1.1267, Val Acc: 72.09%
Training complete!


Train Accuracy,▁▄▆▇█
Train Loss,█▅▃▂▁
Validation Accuracy,▆▁█▃▇
Validation Loss,▁▃▂█▇
epoch+1,▁▁▁▁▁
Train Accuracy,94.1375
Train Loss,0.17913
Validation Accuracy,72.08604
Validation Loss,1.12672
epoch+1,6


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define the device globally
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def evaluate_on_test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    test_loss = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss /= total
    test_acc = 100. * correct / total
    print(f"\nTest Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%")
    return test_loss, test_acc

test_dir = '/kaggle/input/inaturalist/inaturalist_12K/val'
test_loader = test_data_load(test_dir, data_augmentation='No')
# Call the test function
evaluate_on_test(trained_model, test_loader)

Testing: 100%|██████████| 63/63 [00:38<00:00,  1.66it/s]


Test Loss: 0.9777, Test Accuracy: 74.75%


(0.9776781388521194, 74.75)